# Fraud detection

In [19]:
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')

In [20]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import recall_score,f1_score,precision_score
#Visualization
import matplotlib as mpl
import seaborn as sns
from sklearn.metrics import auc
from sklearn import metrics
from imblearn.over_sampling import SMOTE 
from sklearn.metrics import accuracy_score, balanced_accuracy_score, matthews_corrcoef, f1_score, recall_score, precision_score, roc_auc_score
from sklearn.model_selection import KFold
# Import necessary libraries for classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
import pandas as pd
from statistics import mean
import time

In [21]:
df = pd.read_csv('creditcard.csv')
df.info()
df.head() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [22]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [23]:
df.drop(['Time'], axis=1, inplace=True)

In [24]:
df.shape

(284807, 30)

In [25]:
selected_vars =['V4','V14','V10','V16','V9','V12','V11','V17','V7','V21','V18','V8','V1','V3','V6','V2','V27','V5','V26']

In [26]:
df1=df[selected_vars]
Target=df['Class']
df.drop('Class',axis=1, inplace=True)

In [27]:
df1.head()

,V4,V14,V10,V16,V9,V12,V11,V17,V7,V21,V18,V8,V1,V3,V6,V2,V27,V5,V26
0,1.378155,-0.311169,0.090794,-0.470401,0.363787,-0.617801,-0.551600,0.207971,0.239599,-0.018307,0.025791,0.098698,-1.359807,2.536347,0.462388,-0.072781,0.133558,-0.338321,-0.189115
1,0.448154,-0.143772,-0.166974,0.463917,-0.255425,1.065235,1.612727,-0.114805,-0.078803,-0.225775,-0.183361,0.085102,1.191857,0.166480,-0.082361,0.266151,-0.008983,0.060018,0.125895
2,0.379780,-0.165946,0.207643,-2.890083,-1.514654,0.066084,0.624501,1.109969,0.791461,0.247998,-0.121359,0.247676,-1.358354,1.773209,1.800499,-1.340163,-0.055353,-0.503198,-0.139097
3,-0.863291,-0.287924,-0.054952,-1.059647,-1.387024,0.178228,-0.226487,-0.684093,0.237609,-0.108300,1.965775,0.377436,-0.966272,1.792993,1.247203,-0.185226,0.062723,-0.010309,-0.221929
4,0.403034,-1.119670,0.753074,-0.451449,0.817739,0.538196,-0.822843,-0.237033,0.592941,-0.009431,-0.038195,-0.270533,-1.158233,1.548718,0.095921,0.877737,0.219422,-0.407193,0.502292


Final Dataframe

In [28]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 19 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   V4      284807 non-null  float64
 1   V14     284807 non-null  float64
 2   V10     284807 non-null  float64
 3   V16     284807 non-null  float64
 4   V9      284807 non-null  float64
 5   V12     284807 non-null  float64
 6   V11     284807 non-null  float64
 7   V17     284807 non-null  float64
 8   V7      284807 non-null  float64
 9   V21     284807 non-null  float64
 10  V18     284807 non-null  float64
 11  V8      284807 non-null  float64
 12  V1      284807 non-null  float64
 13  V3      284807 non-null  float64
 14  V6      284807 non-null  float64
 15  V2      284807 non-null  float64
 16  V27     284807 non-null  float64
 17  V5      284807 non-null  float64
 18  V26     284807 non-null  float64
dtypes: float64(19)
memory usage: 41.3 MB


# outlier

## OneClassSVM

In [29]:
from sklearn.svm import OneClassSVM

In [30]:
df1_fraud=df1[Target==1]
df1_normal=df1[Target==0]
df1_fraud.shape,df1_normal.shape

((492, 19), (284315, 19))

In [31]:
clf = OneClassSVM()
res = clf.fit_predict(df1_fraud)
np.unique(res)

array([-1,  1], dtype=int64)

In [32]:
df1_normal['Class']=0
df1_fraud['Class']=res

In [33]:
frames = [df1_normal, df1_fraud]
result = pd.concat(frames)

In [34]:
osvm_res=result[result['Class']!=-1]
osvm_out=result[result['Class']==-1]
result[result['Class']==-1].shape

(247, 20)

In [35]:
osvm_t=pd.DataFrame(osvm_res['Class'])
osvm_res.drop(['Class'],axis=1,inplace=True)
osvm_ot=pd.DataFrame(osvm_out['Class'])
osvm_out.drop(['Class'],axis=1,inplace=True)

# Balancing

In [36]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
X = osvm_res.copy()
X.reset_index(drop=True, inplace=True)
y = osvm_t.copy()
y.reset_index(drop=True, inplace=True)

# List of classifiers to evaluate
classifiers = {
    "Logistic Regression": LogisticRegression(random_state=0),
    "Decision Tree": DecisionTreeClassifier(random_state=0),
    "Support Vector Machine": SVC(random_state=0),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "AdaBoost": AdaBoostClassifier(random_state=0),
    "Multi Layer Perceptron": MLPClassifier(random_state=0)
}

# Metrics storage
results = {name: {"f1": [], "acc": [], "recall": [], "precision": [], "AUC": [], "mcc": [], "bal_acc": [], "train_time": []} for name in classifiers}

for name, clf in classifiers.items():
    print(f"Evaluating {name}...")
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]

        # Apply SMOTE
        ros = SMOTE(random_state=0, sampling_strategy=0.0526315789)
        X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
        X_resampled = pd.concat([X_resampled, osvm_out])
        y_resampled = pd.concat([y_resampled, osvm_ot])
        y_resampled[y_resampled == -1] = 1

        # Train the classifier and measure training time
        start_time = time.time()
        clf.fit(X_resampled, y_resampled)
        end_time = time.time()
        train_time = end_time - start_time

        # Store training time
        results[name]["train_time"].append(train_time)

        preds = clf.predict(X_test)
        pred_proba = clf.predict_proba(X_test)[:, 1] if hasattr(clf, "predict_proba") else preds

        # Metrics calculation
        results[name]["f1"].append(f1_score(y_test, preds))
        results[name]["acc"].append(accuracy_score(y_test, preds))
        results[name]["recall"].append(recall_score(y_test, preds))
        results[name]["precision"].append(precision_score(y_test, preds))
        results[name]["AUC"].append(roc_auc_score(y_test, pred_proba))
        results[name]["mcc"].append(matthews_corrcoef(y_test, preds))
        results[name]["bal_acc"].append(balanced_accuracy_score(y_test, preds))

# Print mean metrics for each classifier
for name in classifiers:
    print(f"\n{name} Results:")
    print("Mean ACC: ", mean(results[name]["acc"]), 
          "Mean Precision: ", mean(results[name]["precision"]), 
          "Mean Recall: ", mean(results[name]["recall"]), 
          "Mean F1: ", mean(results[name]["f1"]), 
          "Mean AUC: ", mean(results[name]["AUC"]), 
          "Mean MCC: ", mean(results[name]["mcc"]), 
          "Mean Balanced Accuracy: ", mean(results[name]["bal_acc"]),
          "Mean Training Time (s): ", mean(results[name]["train_time"]))

Evaluating Logistic Regression...
Evaluating Decision Tree...
Evaluating Support Vector Machine...
Evaluating K-Nearest Neighbors...
Evaluating Naive Bayes...
Evaluating AdaBoost...
Evaluating Multi Layer Perceptron...

Logistic Regression Results:
Mean ACC:  0.9992374191734608 Mean Precision:  0.5328449804164089 Mean Recall:  0.9304441962405982 Mean F1:  0.6769339358205464 Mean AUC:  0.9965578840880263 Mean MCC:  0.7034194700396493 Mean Balanced Accuracy:  0.9648703744719689 Mean Training Time (s):  0.43143935203552247

Decision Tree Results:
Mean ACC:  0.9992795895417487 Mean Precision:  0.5500239290495315 Mean Recall:  0.8660819693202317 Mean F1:  0.6721602145131557 Mean AUC:  0.9327367437746394 Mean MCC:  0.6895435809547441 Mean Balanced Accuracy:  0.9327367437746394 Mean Training Time (s):  16.655182886123658

Support Vector Machine Results:
Mean ACC:  0.9996134382906944 Mean Precision:  0.7155665711703082 Mean Recall:  0.9093914130333612 Mean F1:  0.8000837997296303 Mean AUC:  0.